In [1]:
import json
from io import StringIO
from tab_dataset import Cdataset
import ntv_pandas as npd

In [2]:
payload1 = pd.read_json("static_14h12m44s.json", orient="record")
payload2 = pd.read_json("static_14h12m46s.json", orient="record")
payload = pd.concat([payload1, payload2], ignore_index=True).reset_index()
payload = pd.read_json("bulk_evses_payload.json", orient="record").reset_index()

In [3]:
with open("payload_pdc.json", "r") as f:
    data = f.read()
payload = pd.read_json(StringIO(json.loads(data)), orient="record").reset_index()
#payload

In [4]:
payload.columns

Index(['index', 'accessibilite_pmr', 'cable_t2_attache', 'adresse_station',
       'code_insee_commune', 'condition_acces', 'contact_operateur',
       'coordonneesXY', 'date_maj', 'date_mise_en_service', 'gratuit',
       'horaires', 'id_pdc_itinerance', 'id_pdc_local',
       'id_station_itinerance', 'id_station_local', 'implantation_station',
       'nbre_pdc', 'nom_enseigne', 'nom_station', 'num_pdl', 'observations',
       'paiement_acte', 'paiement_cb', 'paiement_autre', 'prise_type_2',
       'prise_type_autre', 'prise_type_chademo', 'prise_type_combo_ccs',
       'prise_type_ef', 'puissance_nominale', 'raccordement', 'reservation',
       'restriction_gabarit', 'station_deux_roues', 'tarification',
       'nom_amenageur', 'nom_operateur', 'siren_amenageur',
       'contact_amenageur', 'telephone_operateur'],
      dtype='object')

In [5]:
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "nom_amenageur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "siren_amenageur"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_amenageur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_operateur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "telephone_operateur"],  "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "num_pdl"],              "link" : "derived" },
         {"fields": ["id_station_itinerance", "id_station_local"],     "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         {"fields": ["id_station_itinerance", "raccordement"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_mise_en_service"], "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" },
         {"fields": ["coordonneesXY", "code_insee_commune"],           "link" : "derived" }
    ]
}

In [6]:
dic_errors = Cdataset(payload).check_relationship(schema)
dic_errors

{'index - id_pdc_itinerance': (),
 'nom_amenageur - id_station_itinerance': (),
 'siren_amenageur - id_station_itinerance': (),
 'contact_amenageur - id_station_itinerance': (),
 'nom_operateur - id_station_itinerance': (),
 'contact_operateur - id_station_itinerance': (),
 'telephone_operateur - id_station_itinerance': (),
 'nom_enseigne - id_station_itinerance': (),
 'coordonneesXY - id_station_itinerance': (),
 'id_station_itinerance - id_pdc_itinerance': (),
 'nom_station - id_station_itinerance': (),
 'implantation_station - id_station_itinerance': (),
 'date_maj - id_station_itinerance': (),
 'num_pdl - id_station_itinerance': (),
 'id_station_local - id_station_itinerance': (),
 'nbre_pdc - id_station_itinerance': (),
 'condition_acces - id_station_itinerance': (),
 'horaires - id_station_itinerance': (),
 'station_deux_roues - id_station_itinerance': (),
 'raccordement - id_station_itinerance': (),
 'date_mise_en_service - id_station_itinerance': (5,
  96,
  145,
  146,
  155,


In [7]:
errors = payload.loc[dic_errors['date_mise_en_service - id_station_itinerance'], ['date_mise_en_service', 'id_station_itinerance', 'id_pdc_itinerance']]
errors['id_station_itinerance'].unique()

array(['FRBMPP646636P', 'FRBMPP202463P', 'FRBMPP633939P',
       'FRBMPP202458P50049568141735', 'FRBMPP269842P50041318179470',
       'FRBMPP647897P'], dtype=object)

In [8]:
errors.loc[errors['id_station_itinerance'] == 'FRBMPP202463P', :]

,date_mise_en_service,id_station_itinerance,id_pdc_itinerance
32,2024-02-22,FRBMPP202463P,FRBMPE9494
476,2025-01-01,FRBMPP202463P,FRBMPE3253


In [9]:
errors.loc[errors['id_station_itinerance'] == 'FRBMPP647897P', :]

,date_mise_en_service,id_station_itinerance,id_pdc_itinerance
245,2025-06-16,FRBMPP647897P,FRBMPE7494
296,2025-06-16,FRBMPP647897P,FRBMPE4231
379,2025-06-16,FRBMPP647897P,FRBMPE1285
417,2025-06-16,FRBMPP647897P,FRBMPE5815
458,2025-06-16,FRBMPP647897P,FRBMPE5392
374,2025-07-15,FRBMPP647897P,FRBMPE9742
